# 5. 셀레니움 활용 웹브라우저 자동화

In [1]:
# from selenium import webdriver
# from bs4 import BeautifulSoup
# import os
# from os.path import join, dirname

# if os.name == 'nt':
#     os_name = 'nt'
# elif os.name == 'mac':
#     os_name = 'mac'
# else:
#     os_name = 'linux'

# if os_name == 'nt':
#     dirver_loc = "d:/chromedriver/chromedriver.exe"
# else:
#     dirver_loc = "/home/download/chromedriver/chromedriver.exe"

### (1) 라이브러리 선언 및 드라이버 설정

In [2]:
# 라이브러리 선언
from selenium import webdriver

# 드라이버 위치 설정
driver_loc = "d:/chromedriver/chromedriver.exe"

# 드라이버 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")

# 헤드리스 설정 시 리눅스 같이 웹화면 없는경우에도 웹페이지 실행가능함
# options.add_argument('headless’)
# options.add_argument("disable-gpu")
# 웹 드라이버 정의
driver = webdriver.Chrome(driver_loc, options=options)
# 웹페이지 파싱 될때까지 최대 3초 기다림
driver.implicitly_wait(3)

### (2) 웹페이지 이동

In [3]:
# URL 정의
baseUrl = "https://sparkkorea.com"
# URL 이동
driver.get(baseUrl)

In [4]:
# 현재 URL 정보 
driver.current_url

'https://sparkkorea.com/'

### (3) 액션대상 요소 탐색

웹 페이지에서 원하는 요소 선택 (우클릭 후 검사) 이후 소스코드 영역에서 우클릭 후 COPY X PATH
(* 클래스 속성으로 등으로도 접근가능)

In [5]:
serchPath = '//*[@id="menu-item-382"]/a'

### (4) 액션 적용하기 (키 입력)

In [6]:
from selenium.webdriver.common.keys import Keys

# URL 정의
googleUrl = 'https://www.google.co.kr'
# URL 이동
driver.get(googleUrl)
# 요소 탐색
searchPath = '//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[2]/input'
driver.find_element_by_xpath(searchPath).send_keys("selenium wiki")
driver.find_element_by_xpath(searchPath).send_keys(Keys.ENTER)

### (4) 액션 적용하기 (클릭)

In [7]:
# URL 정의
sparkUrl = 'https://sparkkorea.com'

# URL 이동
driver.implicitly_wait(3)
driver.get(sparkUrl)

quizBtnXpath = '//*[@id="menu-item-382"]/a'
driver.find_element_by_xpath(quizBtnXpath).click()

### (5) 웹 페이지 소스 가져오기

In [8]:
# 페이지 소스 가져오기 전 대기 (절대시간 1초)
import time
time.sleep(1)

# 현재 페이지 소스 가져오기
html = driver.page_source

import bs4
# BeautifulSoup로 페이지 소스 파싱
bs = bs4.BeautifulSoup(html,"html.parser")

### (6) 페이지 스크랩 및 저장하기

In [9]:
# 퀴즈 정보 스크랩
spQuizTag = bs.find(name="div", attrs={"id":"id_spark_quiz"})

spQuizTagLinks = spQuizTag.findAll("a")

titleList = []
linkList = []

for i in range(0, len(spQuizTagLinks)):
    titleInfo = spQuizTagLinks[0].text
    linkInfo = spQuizTagLinks[0].attrs["href"]
    titleList.append(titleInfo)
    linkList.append(linkInfo)
    
# 데이터프레임 변환
import pandas as pd
finalResult = pd.DataFrame( zip(titleList, linkList) , columns = ["제목", "링크"])

# csv 파일로 저장
finalResult.to_csv("./link_scraping_result.csv",encoding="ms949", index=False)

# 액션 고급

##### 1. 쿠팡접속 메뉴 연속 선택

In [10]:
from selenium.webdriver.common.action_chains import ActionChains
coupangUrl = 'http://www.coupang.com'
driver.get(coupangUrl)

# 카테고리 메뉴 (카테고리 -> 채소 )
mainMenu = '//*[@id="header"]/div/a'
subMenu = '//*[@id="gnbAnalytics"]/ul[1]/li[4]/a'

main = driver.find_element_by_xpath(mainMenu)
sub = driver.find_element_by_xpath(subMenu)

# 메인 이동 후 클릭
ActionChains(driver).move_to_element(main).click(sub).perform()

##### 2. 쿠팡접속 메뉴 연속 선택

In [11]:
# 카테고리 메뉴 (카테고리 -> 식품 -> 축산/계란 -> 게략/알류/가공란)
mainMenu = '//*[@id="header"]/div/a'
subMenu = '//*[@id="gnbAnalytics"]/ul[1]/li[4]/a'
subMenu2 = '//*[@id="gnbAnalytics"]/ul[1]/li[4]/div/div/ul/li[6]/a'
targetMenu = '//*[@id="gnbAnalytics"]/ul[1]/li[4]/div/div/ul/li[6]/div/ul/li[6]/a'
main = driver.find_element_by_xpath(mainMenu)
sub = driver.find_element_by_xpath(subMenu)
sub2 = driver.find_element_by_xpath(subMenu2)
target = driver.find_element_by_xpath(targetMenu)
ActionChains(driver).move_to_element(main).move_to_element(sub).move_to_element(sub2).click(target).perform()

### 액션 적용 시 기다렸다가 클릭하기

In [12]:
# URL 정의
baseUrl = "https://sparkkorea.com"
# URL 이동
driver.get(baseUrl)

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# driver.find_element_by_xpath(mainTitle).is_displayed()

quizMenu = '//*[@id="menu-item-382"]/a'
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, quizMenu)))
driver.find_element_by_xpath(quizMenu).click()

In [13]:
# mainMenu = '//*[@id="site-navigation"]/button'
# subMenu = '//*[@id="menu-item-382"]/a'

# try:
#     # 메인 메뉴가 있을때 메인메뉴 클릭하고 퀴즈 메뉴 선택
#     driver.find_element_by_xpath(mainMenu).click()
#     driver.find_element_by_xpath(subMenu).click()
# except Exception as e:
#     # 메인 메뉴 없는 경우 퀴즈 메뉴 바로 선택
#     print("menu not founded")
#     driver.find_element_by_xpath(subMenu).click()

### [실습] [selenium 활용] sparkkorea.com 사이트내 
퀴즈 페이지 에서
스파크 퀴즈  퀴즈이름 및 링크정보를 스크랩 하세요

In [14]:
divScrape = bs.find(name="div",attrs={"class":"class_spark_quiz"})

In [15]:
aTags = divScrape.findAll(name="a")

In [16]:
linkList = []
titleList = []
for i in range(0, len(aTags)):
    linkList.append(aTags[i].attrs["href"])
    titleList.append(aTags[i].text)

In [17]:
import pandas as pd
finalResult = pd.DataFrame(zip(titleList, linkList),
            columns = ["spark퀴즈 타이틀", "spark퀴즈 링크"] )